# Fitting graph embeddings to the network of interactions

In [1]:
from gensim.models.keyedvectors import KeyedVectors

In [2]:
vectors_file_vec = 'encodings/italy_embeddings.tsv'

In [3]:
cantidad = 198613
vectors = KeyedVectors.load_word2vec_format(vectors_file_vec, limit=cantidad)

In [4]:
import os, sys
import csv
import numpy as np

The file 'interactions.dat' is a row file of edges, and each edge is a pair <uids,uidd>

In [6]:
file_X = open("bot_detection/Italy/interactions.dat",'r')
reader_csv = csv.reader(file_X, delimiter=" ")

In [7]:
neighbors = dict()
for line in reader_csv:
    node_from = str(line[0])
    node_to = str(line[1])
    if node_from not in neighbors:
        neighbors[node_from] = list()
        neighbors[node_from].append(node_to)
    else:
        neighbors[node_from].append(node_to)
        
file_X.close()

In [35]:
from copy import deepcopy

def retrofit(X, neighbors, numIters):
    X_ = deepcopy(X)
    n = X.shape[0]
    for it in range(numIters):
        for i in range(n):
            v = idx2v[i]
            if v in neighbors:
                nns = neighbors[v]
                numNeighbors = len(nns)
                if numNeighbors > 0:
                    newVec = numNeighbors * X[i,]
                    for u in nns:
                        if u in v2idx:
                            ni = v2idx[u]
                            newVec += X_[int(ni),]
                        else:
                            newVec += X[i,]
                        X_[i,] = newVec/(2*numNeighbors)
                
    return X_


In [36]:
X = np.empty((cantidad,100))

In [37]:
v2idx = dict()
idx2v = dict()
i = 0
for v in vectors.vocab:
    v2idx[v] = i
    idx2v[i] = str(v)
    X[i,] = vectors[v]
    i += 1

In [38]:
X_ = retrofit(X, neighbors, 10)

In [46]:
n = X_.shape[0]

In [49]:
file = open('keys.dat', 'w')
for i in range(n):
    v = idx2v[i]
    file.write(v+'\n')

file.close()

In [50]:
np.savetxt('interaction_vectors', X_)

- paste keys.dat interaction_vectors > interactions_italy.tsv
- vi interaction_italy.tsv: :%s/\t/ /g (wq)